In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import nltk


In [ ]:
text1 = input('Please enter the results of the OCR here:')

In [ ]:
text2 = input('Please enter the results of the OCR here:')

In [ ]:
text3 = input('Please enter the results of the OCR here:')

In [ ]:
def getPhoneNumber(text):
    #Writing regex to grab the first 5 characters before the phone number to ensure we do not grab a fax number
    phone_pattern = re.compile(r'((?:.{5})(?:\+1\s)?[\(]?\d{3}[\)|-]\s?\d{3}[-|\s]\d{4})')
    #matching the pattern and grabbing the second capture group which is the number
    number =re.findall(phone_pattern, text)
    #only selecting the numbers that are not labeled as a fax.
    number = [x[5:] for x in number if 'Fax' not in x][0]
    #cleaning the number by removing any non-digits
    number = re.sub(r'\D','',number)
    return('Phone: '+number)

In [ ]:
def getEmail(text):
    #simple regex for email limiting the size of each component to 255 out of habit as regex's are expensive. 
    emailPattern = re.compile(r'\s((?:\w|\.){1,255}@\w{1,255}\.\w{1,255})')
    email = re.findall(emailPattern, text)
    return('Email: '+email[0].strip())

In [ ]:
getEmail(text3)

In [ ]:
'''
I wanted to use a package like NLTK or SPACY to extact the names,
but both required specific large downloads and given that I cannot be sure of the system my code 
will be running on I implememted a solution using the Social Security Names.  I took a random guess and 
picked 1980 as the best year to compare for given names and then used a regex to extact the last name.  If 
this was going to be used in the wild I would want to download a more comprehensive name corpus or train an 
ML model
'''

last = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.all.last?#').text
first_f = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.female.first?#').text
first_m = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.male.first?#').text
first = ' '.join([first_f, first_m])
first = re.sub('\d',' ', first)

In [ ]:
tokens = text3.split(' ')
for word in tokens:
    if (word.upper() in first) & :


# Code With Widget

In [1]:
import ipywidgets as wi
import nltk
import re

In [6]:
style = {'description_width': 'initial'}
test = wi.Textarea(description='Enter results from OCR here:',
                   layout=wi.Layout(width='70%', height='150px'),
                   style= style)
test

Textarea(value='', description='Enter results from OCR here:', layout=Layout(height='150px', width='70%'), sty…

In [7]:
data = test.value

In [8]:
def parse_person(line):
    names = []
    #using the NTLK parts of speach model to check for people.
    for i, chunk in enumerate(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(line)))):
        if type(chunk) == nltk.tree.Tree:
            if chunk.label() == 'PERSON':
                names.append(chunk[0][0])
    '''
    The above code works great for english names.  However it struggled to identify the non-anglo surname 'Haung'
    therefore I added this code to grab the second word after an NLTK identified person.  
    to solve the non-anglo problem effecently you would want to retrain a word net underling NLTK to include 
    a more comprehensive list of global names. 
    
    A potential bug in this code is that it would extract a company such as "Kellys Carpets" as a name.
    '''
    if len(names)==1:
        names.append(line.split(' ')[i])
    return ' '.join(names)
        
def getName(data):
    data = data.split('\n')
    for line in data:
        person = parse_person(line)
        if person:
            return('Name: '+person)    
        
def getPhoneNumber(text):
    #removing the newline character to allow for the functions to be used in both the GUI and non-GUI code
    text = text.replace('\n','')
    #Writing regex to grab the first 5 characters before the phone number to ensure we do not grab a fax number
    phone_pattern = re.compile(r'((?:.{5})(?:\+1\s)?[\(]?\d{3}[\)|-]\s?\d{3}[-|\s]\d{4})')
    #matching the pattern and grabbing the second capture group which is the number
    number =re.findall(phone_pattern, text)
    #only selecting the numbers that are not labeled as a fax.
    number = [x[5:] for x in number if 'Fax' not in x][0]
    #cleaning the number by removing any non-digits
    number = re.sub(r'\D','',number)
    return('Phone: '+number)


def getEmail(text):
    #removing the newline character to allow for the functions to be used in both the GUI and non-GUI code
    text = text.replace('\n',' ')
    #simple regex for email limiting the size of each component to 255 out of habit as regex's are expensive. 
    emailPattern = re.compile(r'\s((?:\w|\.){1,255}@\w{1,255}\.\w{1,255})')
    email = re.findall(emailPattern, text)
    
    return('Email: '+email[0].strip())

In [9]:
print(getName(data))
print(getPhoneNumber(data))
print(getEmail(data))

Name: Arthur Wilson
Phone: 17035551259
Email: awilson@abctech.com
